In [2]:
from pyspark.sql import SparkSession

In [3]:
# Initialize SparkSession
spark = (SparkSession.builder
        .appName("cs744")
        .master("spark://master:7077")
        .config("spark.driver.memory", "20g")
        .config("spark.executor.memory", "20g")
        .config("spark.executor.cores", "4")
        .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/30 22:51:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
holidays = spark.read.csv("hdfs://nn:9000/holidays2.csv", inferSchema=True, header=True)

In [16]:
holidays.show()

+----------+--------------------+
|      date|             holiday|
+----------+--------------------+
|01/01/2013|      New Year's Day|
|01/01/2014|      New Year's Day|
|01/01/2015|      New Year's Day|
|01/01/2016|      New Year's Day|
|01/01/2018|      New Year's Day|
|01/01/2019|      New Year's Day|
|01/01/2020|      New Year's Day|
|01/01/2021|      New Year's Day|
|01/02/2012|      New Year's Day|
|01/02/2017|      New Year's Day|
|01/15/2018|Birthday of Marti...|
|01/16/2012|Birthday of Marti...|
|01/16/2017|Birthday of Marti...|
|01/17/2011|Birthday of Marti...|
|01/17/2022|Birthday of Marti...|
|01/18/2016|Birthday of Marti...|
|01/18/2021|Birthday of Marti...|
|01/19/2015|Birthday of Marti...|
|01/20/2014|Birthday of Marti...|
|01/20/2020|Birthday of Marti...|
+----------+--------------------+
only showing top 20 rows



In [10]:
holiday.limit(5).toPandas()

,date,holiday
0,01/01/2013,New Year's Day
1,01/01/2014,New Year's Day
2,01/01/2015,New Year's Day
3,01/01/2016,New Year's Day
4,01/01/2018,New Year's Day


In [11]:
calls = spark.read.csv("hdfs://nn:9000/sf.csv", inferSchema=True, header=True)

In [12]:
calls.limit(3).toPandas()

25/01/30 22:56:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,...,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
0,221210313,E36,22054955,Outside Fire,05/01/2022,04/30/2022,05/01/2022 02:58:25 AM,05/01/2022 02:59:15 AM,05/01/2022 02:59:25 AM,05/01/2022 03:01:06 AM,...,Fire,1,ENGINE,1,2,5,Hayes Valley,221210313-E36,POINT (-122.42316555403964 37.77781524520032),9
1,220190150,E29,22008871,Alarms,01/19/2022,01/18/2022,01/19/2022 01:42:12 AM,01/19/2022 01:44:13 AM,01/19/2022 01:44:28 AM,01/19/2022 01:46:47 AM,...,Alarm,1,ENGINE,1,3,10,Potrero Hill,220190150-E29,POINT (-122.39469970274361 37.76460987856451),26
2,211233271,T07,21053032,Alarms,05/03/2021,05/03/2021,05/03/2021 09:28:12 PM,05/03/2021 09:28:12 PM,05/03/2021 09:28:17 PM,05/03/2021 09:29:10 PM,...,Alarm,1,TRUCK,2,2,9,Mission,211233271-T07,POINT (-122.42057572093252 37.76418194637148),20


# Filtering

In [13]:
from pyspark.sql.functions import expr

In [19]:
holidays_14 = holidays.filter(expr("date like '%2014'"))

In [21]:
holidays_14_birth = holidays_14.filter(expr("holiday like '%Birthday%'"))

In [22]:
holidays_14_birth.show()

+----------+--------------------+
|      date|             holiday|
+----------+--------------------+
|01/20/2014|Birthday of Marti...|
|02/17/2014|Washington's Birt...|
+----------+--------------------+



In [23]:
holidays_14_birth.count()

2

# Selecting Columns

In [27]:
calls_call_date = calls.select("Call Date")

In [30]:
calls_call_date.count()

6016056

# Grouping

In [31]:
# SELECT SUM(`Number of Alarms`) as "Total Alarms"
# FROM calls
# GROUP BY `Call Type`
# WHERE `Call Date` = '01/01/2014'

In [33]:
from pyspark.sql.functions import sum

In [32]:
calls_date = calls.filter(expr("`Call Date` = '01/01/2014'"))

In [34]:
calls_grouped = calls_date.groupBy("Call Type").agg(sum("Number of Alarms").alias("Num_Alarms"))

In [36]:
calls_grouped.show()

[Stage 20:====================================================>   (16 + 1) / 17]

+--------------------+----------+
|           Call Type|Num_Alarms|
+--------------------+----------+
|              Alarms|        82|
|      Structure Fire|       164|
|Elevator / Escala...|         1|
|Citizen Assist / ...|        10|
|        Vehicle Fire|         2|
|               Other|        25|
|        Outside Fire|        35|
|   Traffic Collision|        54|
|Gas Leak (Natural...|         7|
|        Water Rescue|        13|
|    Medical Incident|       818|
|          Fuel Spill|         5|
|Smoke Investigati...|         2|
+--------------------+----------+



# Caching

In [37]:
calls_high = calls_grouped.filter(expr("Num_Alarms > 100"))

In [38]:
calls_high.show()

[Stage 23:=================================================>      (15 + 2) / 17]

+----------------+----------+
|       Call Type|Num_Alarms|
+----------------+----------+
|  Structure Fire|       164|
|Medical Incident|       818|
+----------------+----------+



In [39]:
calls_grouped.cache()

DataFrame[Call Type: string, Num_Alarms: bigint]

In [40]:
calls_high = calls_grouped.filter(expr("Num_Alarms > 100"))

In [41]:
calls_high.show()

+----------------+----------+
|       Call Type|Num_Alarms|
+----------------+----------+
|  Structure Fire|       164|
|Medical Incident|       818|
+----------------+----------+



In [42]:
calls_high = calls_grouped.filter(expr("Num_Alarms > 100"))
calls_high.show()

+----------------+----------+
|       Call Type|Num_Alarms|
+----------------+----------+
|  Structure Fire|       164|
|Medical Incident|       818|
+----------------+----------+



# Join

In [44]:
jdf = holidays.join(calls, holidays["date"] == calls["Call Date"], "inner")

In [45]:
jdf.limit(3).toPandas()

,date,holiday,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,...,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
0,01/17/2022,"Birthday of Martin Luther King, Jr.",220173023,67,22008351,Medical Incident,01/17/2022,01/17/2022,01/17/2022 09:23:31 PM,01/17/2022 09:24:26 PM,...,Potentially Life-Threatening,1,MEDIC,2,10,10,Bayview Hunters Point,220173023-67,POINT (-122.39096789028375 37.73401387414893),1
1,01/17/2022,"Birthday of Martin Luther King, Jr.",220170811,B02,22008104,Alarms,01/17/2022,01/17/2022,01/17/2022 09:28:15 AM,01/17/2022 09:29:17 AM,...,Alarm,1,CHIEF,2,2,9,Mission,220170811-B02,POINT (-122.4207461905871 37.765041354178884),20
2,07/05/2021,Independence Day,211862605,E07,21079789,Medical Incident,07/05/2021,07/05/2021,07/05/2021 05:19:43 PM,07/05/2021 05:24:46 PM,...,Non Life-threatening,1,ENGINE,1,2,9,Mission,211862605-E07,POINT (-122.42173999514011 37.76411273839148),20


In [47]:
cdf = jdf.filter(expr("`Call Date` = '01/01/2014'"))

In [48]:
cdf.count()

1218

# Partitioning

In [51]:
calls.count()

6016056

In [52]:
calls.rdd.getNumPartitions()

17

In [53]:
df = calls.sample(0.01)

In [55]:
%%timeit
df.count()

[Stage 85:====================================================>   (16 + 1) / 17]

3.39 s ± 506 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Caching

In [56]:
df = calls.sample(0.01)
df.cache()

DataFrame[Call Number: int, Unit ID: string, Incident Number: int, Call Type: string, Call Date: string, Watch Date: string, Received DtTm: string, Entry DtTm: string, Dispatch DtTm: string, Response DtTm: string, On Scene DtTm: string, Transport DtTm: string, Hospital DtTm: string, Call Final Disposition: string, Available DtTm: string, Address: string, City: string, Zipcode of Incident: int, Battalion: string, Station Area: string, Box: string, Original Priority: string, Priority: string, Final Priority: int, ALS Unit: boolean, Call Type Group: string, Number of Alarms: int, Unit Type: string, Unit sequence in call dispatch: int, Fire Prevention District: string, Supervisor District: string, Neighborhooods - Analysis Boundaries: string, RowID: string, case_location: string, Analysis Neighborhoods: int]

In [57]:
df.count()

60572

In [58]:
%%timeit
df.count()

127 ms ± 6.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Repartitioning (without caching)

In [60]:
df.unpersist()

DataFrame[Call Number: int, Unit ID: string, Incident Number: int, Call Type: string, Call Date: string, Watch Date: string, Received DtTm: string, Entry DtTm: string, Dispatch DtTm: string, Response DtTm: string, On Scene DtTm: string, Transport DtTm: string, Hospital DtTm: string, Call Final Disposition: string, Available DtTm: string, Address: string, City: string, Zipcode of Incident: int, Battalion: string, Station Area: string, Box: string, Original Priority: string, Priority: string, Final Priority: int, ALS Unit: boolean, Call Type Group: string, Number of Alarms: int, Unit Type: string, Unit sequence in call dispatch: int, Fire Prevention District: string, Supervisor District: string, Neighborhooods - Analysis Boundaries: string, RowID: string, case_location: string, Analysis Neighborhoods: int]

In [62]:
df = calls.sample(0.01).repartition(1)

In [63]:
%%timeit
df.count()

[Stage 598:===================================================>   (16 + 1) / 17]

3.94 s ± 870 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Repartitioning (with caching)

In [64]:
df = calls.sample(0.01).repartition(1)
df.cache()

DataFrame[Call Number: int, Unit ID: string, Incident Number: int, Call Type: string, Call Date: string, Watch Date: string, Received DtTm: string, Entry DtTm: string, Dispatch DtTm: string, Response DtTm: string, On Scene DtTm: string, Transport DtTm: string, Hospital DtTm: string, Call Final Disposition: string, Available DtTm: string, Address: string, City: string, Zipcode of Incident: int, Battalion: string, Station Area: string, Box: string, Original Priority: string, Priority: string, Final Priority: int, ALS Unit: boolean, Call Type Group: string, Number of Alarms: int, Unit Type: string, Unit sequence in call dispatch: int, Fire Prevention District: string, Supervisor District: string, Neighborhooods - Analysis Boundaries: string, RowID: string, case_location: string, Analysis Neighborhoods: int]

In [65]:
df.count()

60171

In [66]:
%%timeit
df.count()

52.2 ms ± 3.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Explain

In [67]:
"""
SELECT SUM(`Number of Alarms`)
FROM holidays JOIN calls on holidays.date = calls.`Call Date`
WHERE lower(holiday) like '%birthday%'
"""

"\nSELECT SUM(`Number of Alarms`)\nFROM holidays JOIN calls on holidays.date = calls.`Call Date`\nWHERE lower(holiday) like '%birthday%'\n"

In [68]:
joined_df = holidays.join(calls, holidays['date'] == calls['Call Date'], "inner")
birth_df = joined_df.filter(expr("lower(holiday) like '%birthday%'"))
birth_df = birth_df.agg(sum("Number of Alarms"))

In [69]:
birth_df.count()

1

In [70]:
birth_df.show()

[Stage 770:===================================================>   (16 + 1) / 17]

+---------------------+
|sum(Number of Alarms)|
+---------------------+
|                18646|
+---------------------+



In [71]:
birth_df.explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (12)
+- HashAggregate (11)
   +- Exchange (10)
      +- HashAggregate (9)
         +- Project (8)
            +- BroadcastHashJoin Inner BuildLeft (7)
               :- BroadcastExchange (4)
               :  +- Project (3)
               :     +- Filter (2)
               :        +- Scan csv  (1)
               +- Filter (6)
                  +- Scan csv  (5)


(1) Scan csv 
Output [2]: [date#209, holiday#210]
Batched: false
Location: InMemoryFileIndex [hdfs://nn:9000/holidays2.csv]
PushedFilters: [IsNotNull(holiday), IsNotNull(date)]
ReadSchema: struct<date:string,holiday:string>

(2) Filter
Input [2]: [date#209, holiday#210]
Condition : ((isnotnull(holiday#210) AND Contains(lower(holiday#210), birthday)) AND isnotnull(date#209))

(3) Project
Output [1]: [date#209]
Input [2]: [date#209, holiday#210]

(4) BroadcastExchange
Input [1]: [date#209]
Arguments: HashedRelationBroadcastMode(List(input[0, string, true]),false), [plan_id=8159]

(5) Scan cs